In [1]:
from danial import model
from danial import dataloader


import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torchvision.transforms import GaussianBlur, Normalize, ToTensor
import torchvision.transforms.functional as TF
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np


In [2]:
mod = model.Model()

/Users/dania/code/fyp/MHNet/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dania/code/fyp/MHNet/vengeance/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Load pretrained weights from models/first600 (robust to common checkpoint formats)
checkpoint_path = "models/first600/best.pth"
try:
    ckpt = torch.load(checkpoint_path, map_location='cpu')
    # checkpoint may be a dict with 'state_dict' or may be the state_dict itself
    if isinstance(ckpt, dict) and 'state_dict' in ckpt:
        state = ckpt['state_dict']
    elif isinstance(ckpt, dict) and all(not isinstance(v, dict) for v in ckpt.values()):
        # likely already a state_dict-like mapping
        state = ckpt
    else:
        state = ckpt
except Exception as e:
    print(f"Failed to load checkpoint {checkpoint_path}: {e}")
    state = None
if state is not None:
    from collections import OrderedDict
    new_state = OrderedDict()
    for k, v in state.items():
        # remove DataParallel/DistributedDataParallel 'module.' prefix if present
        new_k = k.replace('module.', '') if k.startswith('module.') else k
        new_state[new_k] = v
    try:
        mod.load_state_dict(new_state, strict=False)
        print(f"Loaded weights from {checkpoint_path}")
    except Exception as e:
        print(f"Failed to load state dict into model: {e}")
mod.eval()



Loaded weights from models/first600/best.pth


Model(
  (backbone): VGGFeatureExtractor(
    (block1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (block2): Sequential(
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (block3): Sequential(
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3,

In [4]:
test = dataloader.load_image("assets/camo-sold-d.png", target_size = (224,224))
with torch.no_grad():
    binary_pred, boundary_pred, fused_feat = mod(test)

avg_map = fused_feat.mean(dim=1)  # [1, 7, 7]

plt.imshow(avg_map[0].detach().cpu(), cmap='jet')
plt.title("Averaged Feature Map")
plt.colorbar()
plt.show()

avg_map = fused_feat.mean(dim=1, keepdim=True)  # [1, 1, 7, 7]

# 1️⃣ Upsample to higher resolution (e.g., 224x224)
upsampled = F.interpolate(avg_map, size=(448,448), mode='bilinear', align_corners=True)

# 2️⃣ Apply Gaussian blur
blur = GaussianBlur(kernel_size=(7,7), sigma=(1.0,1.0))
smoothed = blur(upsampled)

# 3️⃣ Show result
plt.imshow(smoothed[0, 0].detach().cpu(), cmap='jet')  # using 'jet' for heatmap look
plt.title("Smoothed & Upsampled Feature Map")
plt.colorbar()
plt.show()


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 112 but got size 56 for tensor number 1 in the list.

In [5]:
print(binary_pred.shape, boundary_pred.shape, fused_feat.shape)

torch.Size([1, 5]) torch.Size([1, 4]) torch.Size([1, 64, 7, 7])


In [6]:
from PIL import Image
import torchvision.transforms as transforms

def transgender():
    # Load your RGB PNG and convert to grayscale
    gt_image = Image.open('assets/COD10K-CAM-3-Flying-61-Katydid-4237.png').convert('L')  # 'L' = grayscale

    # Convert to tensor [1, 1, 224, 224]
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    gt_tensor = transform(gt_image).unsqueeze(0)  # Add batch dimension
    return gt_tensor
    # print(gt_tensor.shape)
    # # Now both are [1, 1, 224, 224] - calculate metrics!
    # metrics = ImageMetrics(gt_tensor, model_output, metric_type='reconstruction')
    # print(metrics)

gt = transgender()
print(gt.shape)
print(smoothed.shape)



torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])


In [7]:
import torch
import torch.nn.functional as F
import numpy as np

def rgb_to_grayscale(rgb_tensor):
    """Convert RGB image to grayscale using luminosity method."""
    if rgb_tensor.shape[1] == 3:  # [B, C, H, W]
        weights = torch.tensor([0.299, 0.587, 0.114], device=rgb_tensor.device).view(1, 3, 1, 1)
        gray = (rgb_tensor * weights).sum(dim=1, keepdim=True)
    elif rgb_tensor.shape[-1] == 3:  # [B, H, W, C]
        weights = torch.tensor([0.299, 0.587, 0.114], device=rgb_tensor.device)
        gray = (rgb_tensor * weights).sum(dim=-1, keepdim=True)
        gray = gray.permute(0, 3, 1, 2)  # [B, 1, H, W]
    else:
        raise ValueError(f"Expected 3 channels, got shape {rgb_tensor.shape}")
    return gray

def gaussian_window(window_size=11, sigma=1.5, device='cpu'):
    """Create a 2D Gaussian kernel."""
    coords = torch.arange(window_size, dtype=torch.float32) - window_size // 2
    g = torch.exp(-(coords**2) / (2 * sigma**2))
    g = g / g.sum()
    gauss = g[:, None] * g[None, :]
    return gauss.to(device)

class ImageMetrics:
    """Compute reconstruction metrics with Gaussian-weighted SSIM and precision."""
    
    def __init__(self, gt, pred):
        # Convert to tensor if needed
        if not torch.is_tensor(gt):
            gt = torch.tensor(gt)
        if not torch.is_tensor(pred):
            pred = torch.tensor(pred)
        
        # Convert RGB to grayscale if needed
        if gt.shape[1] == 3 and pred.shape[1] == 1:
            gt = rgb_to_grayscale(gt)
        elif gt.shape[1] == 1 and pred.shape[1] == 3:
            pred = rgb_to_grayscale(pred)
        
        # Validate shapes
        if gt.shape != pred.shape:
            raise ValueError(f"Shape mismatch! gt: {gt.shape}, pred: {pred.shape}")
        
        self.gt = gt.float()
        self.pred = pred.float()
        
        # Compute metrics
        self._calculate_metrics()
    
    def _calculate_metrics(self):
        # MSE
        self.mse = F.mse_loss(self.pred, self.gt).item()
        
        # MAE
        self.mae = F.l1_loss(self.pred, self.gt).item()
        
        # PSNR
        max_pixel = 1.0 if self.gt.max() <= 1.0 else 255.0
        self.psnr = 10 * np.log10((max_pixel ** 2) / self.mse) if self.mse > 0 else float('inf')
        
        # Correlation
        gt_flat = self.gt.flatten()
        pred_flat = self.pred.flatten()
        self.correlation = torch.corrcoef(torch.stack([gt_flat, pred_flat]))[0, 1].item()
        
        # Gaussian-weighted SSIM
        self.ssim = self._calculate_gaussian_ssim()
        
        # "Precision" for continuous values (like ratio of correctly predicted intensity)
        eps = 1e-8
        self.precision = (torch.min(self.pred, self.gt).sum() / (self.pred.sum() + eps)).item()
    
    def _calculate_gaussian_ssim(self, window_size=11, sigma=1.5):
        """Compute SSIM using Gaussian window."""
        C1 = 0.01 ** 2
        C2 = 0.03 ** 2
        
        device = self.gt.device
        window = gaussian_window(window_size, sigma, device).unsqueeze(0).unsqueeze(0)
        
        mu1 = F.conv2d(self.gt, window, padding=window_size//2, groups=1)
        mu2 = F.conv2d(self.pred, window, padding=window_size//2, groups=1)
        
        mu1_sq = mu1 ** 2
        mu2_sq = mu2 ** 2
        mu1_mu2 = mu1 * mu2
        
        sigma1_sq = F.conv2d(self.gt**2, window, padding=window_size//2, groups=1) - mu1_sq
        sigma2_sq = F.conv2d(self.pred**2, window, padding=window_size//2, groups=1) - mu2_sq
        sigma12 = F.conv2d(self.gt*self.pred, window, padding=window_size//2, groups=1) - mu1_mu2
        
        ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2)) / ((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
        return ssim_map.mean().item()
    
    def __str__(self):
        return (
            f"{'='*50}\n"
            f"IMAGE RECONSTRUCTION METRICS\n"
            f"{'='*50}\n"
            f"MSE (Mean Squared Error):     {self.mse:.6f}\n"
            f"MAE (Mean Absolute Error):    {self.mae:.6f}\n"
            f"PSNR (Peak Signal-to-Noise):  {self.psnr:.2f} dB\n"
            f"SSIM (Gaussian-weighted):     {self.ssim:.4f}\n"
            f"Correlation Coefficient:      {self.correlation:.4f}\n"
            f"Precision (continuous):       {self.precision:.4f}"
        )
    
    def to_dict(self):
        return {
            'mse': self.mse,
            'mae': self.mae,
            'psnr': self.psnr,
            'ssim': self.ssim,
            'correlation': self.correlation,
            'precision': self.precision
        }


metrics = ImageMetrics(gt, smoothed)
print(metrics)


IMAGE RECONSTRUCTION METRICS
MSE (Mean Squared Error):     0.076334
MAE (Mean Absolute Error):    0.091663
PSNR (Peak Signal-to-Noise):  11.17 dB
SSIM (Gaussian-weighted):     0.5101
Correlation Coefficient:      0.2813
Precision (continuous):       0.1300


In [5]:
print(test.shape)

torch.Size([1, 3, 512, 512])


In [13]:
from PIL import Image

def load_resize_show(image_path, size=(224,224)):
    """
    Loads an image from the given path, resizes it to `size`, and displays it.
    
    Args:
        image_path (str): Path to the image file.
        size (tuple): Target size as (width, height). Default is (512, 512).
    """
    # Load the image
    img = Image.open(image_path)
    print(img.size)
    
    # Resize the image
    img_resized = img.resize(size)
    
    # Show the image
    img_resized.show()

# Example usage
load_resize_show("assets/COD10K-CAM-1-Aquatic-1-BatFish-2.png")


(800, 533)
